### Notebook to parse available run and baval files for each particiant

In [ ]:
import pandas as pd
from bids import BIDSLayout
import seaborn as sns
import numpy as np
import json
import shutil

### Path

In [ ]:
dataset_root = "/home/md_lab/projects/ylopd/data/ylopd/"
bids_dir = f"{dataset_root}bids/"
proc_dir = f"{dataset_root}proc/bids_filter_tractoflow/"

### Create layout

In [ ]:
layout = BIDSLayout(bids_dir)
participants = layout.get_subjects()
sessions = layout.get_sessions()

print(f"n_participants: {len(participants)}, sessions: {sessions}")

### Typical bids_filter used as template with participant specific run_id

In [ ]:
bids_filter_template = {
                            "t1w": {
                                "datatype": "anat",
                                "session": "01",
                                "run": "1",
                                "suffix": "T1w"
                            },
                            "dwi": {
                                "session": "01",                                                          
                                "suffix": "dwi"
                            }
}

def get_bvals(bval_file):
    bval_d = bval_file.dirname
    bval_f = bval_file.filename
    bval_path = f"{bval_d}/{bval_f}"
    bvals_str = pd.read_csv(bval_path,header=None)
    bval_num = [int(b) for b in bvals_str.values[0][0].split(" ")]
    return bval_f, bvals_str, bval_num 

### Query available bvals per participant and write bids_filter with highest number of bval

In [ ]:
dwi_bval_df = pd.DataFrame(columns=["participant_id","session","filename","bvals", "max_bval"])
dwi_file_count_df = pd.DataFrame(columns=["participant_id","session","n_dwi","n_bval","n_bvec","run_id_to_use","run_filename_to_use"])

i = 0
for p in participants:
    for s in sessions: 
        # query participant files
        dwi_files = layout.get(subject=p, session=s, suffix="dwi", extension='nii.gz')
        bval_files = layout.get(subject=p, session=s, suffix="dwi", extension='bval')
        bvec_files = layout.get(subject=p, session=s, suffix="dwi", extension='bvec')
    
        if len(dwi_files) > 0:
            # identify run with the max bvals
            _df = pd.DataFrame(columns=["participant_id", "session", "filename", "dir", "bvals", "max_bval"])
            for b, bval_file in enumerate(bval_files):
                if "dir-AP" in bval_file.filename:
                    bval_dir = "AP"
                else:
                    bval_dir = "PA"                    

                bval_f, bvals_str, bval_num = get_bvals(bval_file)
                max_bval = np.max(bval_num)
                _df.loc[b] = [p, s, bval_f, bval_dir, bvals_str, max_bval]
            
            AP_df = _df[_df["dir"] == "AP"]
            max_bval_across_runs = AP_df["max_bval"].max()
            AP_df_max_bval_across_run = AP_df[AP_df["max_bval"] == max_bval_across_runs]

            file_with_max_bval = AP_df_max_bval_across_run.sort_values("filename",ascending=True)["filename"].values[0]
            max_run_id = file_with_max_bval.rsplit("_",2)[1].split("-")[1]

            # Append participant specific counts
            dwi_file_count_df.loc[i] = [p, s, len(dwi_files), len(bval_files), len(bvec_files), max_run_id, file_with_max_bval]        
            
            # Append participant and run specific info
            dwi_bval_df = pd.concat([dwi_bval_df,_df])

            # Copy PA file to match AP file's run_id
            if max_run_id != "1":
                print(f"For sub-{p}_ses-{s} making a copy to PA dwi, bval, and bvecs to match selected AP run_id: {max_run_id}")
                bval_d = bval_file.dirname
                for suffix in ["bval","bvec","json","nii.gz"]:
                    bval_f_orig = f"sub-{p}_ses-{s}_dir-PA_run-1_dwi.{suffix}"
                    orig_PA_path = f"{bval_d}/{bval_f_orig}"
                    bval_f_copy = f"sub-{p}_ses-{s}_dir-PA_run-{max_run_id}_dwi.{suffix}"
                    copy_PA_path = f"{bval_d}/{bval_f_copy}"
                    shutil.copyfile(orig_PA_path, copy_PA_path)

            # Write custom_bids_filter
            bids_filter_template["dwi"]["run"] = str(max_run_id)
            bids_filter_path = f"{proc_dir}/{p}_{s}_bids_filter.json"
            with open(bids_filter_path, "w") as outfile:
                json.dump(bids_filter_template, outfile, indent = 4)

            i+=1

        else:
            print(f"No DWI files found for {p}, ses-{s}")
dwi_file_count_df.head()

In [ ]:
dwi_file_count_df.tail()

In [ ]:
dwi_bval_df[dwi_bval_df["participant_id"]=="YLOPD100"]

In [ ]:
plot_df = dwi_file_count_df.melt(
    id_vars=["participant_id","session","run_id_to_use","run_filename_to_use"],
    var_name="filetype", 
    value_name="count")

sns.set(font_scale=2)
plot_kind = "point" # "violin", "boxen", "point", "bar", or “strip”
with sns.axes_style("whitegrid"):
    g = sns.catplot(x="participant_id", y="count", hue="filetype", kind="point", data=plot_df, aspect=6)
    g.tick_params(axis='x', rotation=90, labelsize=6)



In [ ]:
dwi_file_count_df.groupby(["n_dwi"]).count()

In [ ]:
dwi_file_count_df[dwi_file_count_df["n_bval"]==9]